# This is a basic RNN model used to test dataset preperation and model evaluation

## Imports

In [1]:
import torch.nn as nn
import torch
import full_iri_dataset_generator as iri
from training_loop import train_model

## Constants

- `SEQUENCE_LENGTH` is the number of historical measurements before the target element to provide to the model
- `NUM_FREATURES_PER_SAMPLE` is how many details each measurement has. `IRI-only` has 3: left_iri, right_iri, and time_since_first_measurement
- `NUM_LAYERS` is the number of RNN layers to use

In [24]:
SEQUENCE_LENGTH = 10
NUM_FEATURES_PER_SAMPLE = 6
NUM_HEADS = 1
NUM_LAYERS = 12

## Dataset Preperation

Load train and test datasets

In [3]:
train, test = iri.load_iri_datasets(path="../training_data/final_data.parquet",
                                    construction_path="../training_data/construction_data.parquet",
                                    seq_length=SEQUENCE_LENGTH)

## Model Definition

Here a basic RNN classifier model is defined.

1. Data is flattened
2. RNN layers process data and modify hidden state
3. final layer maps hidden state to 3 predicted probilities
4. outputs are scaled using a logsoftmax function

In [25]:
class Transformer(nn.Module):
    def __init__(self):
        super(Transformer, self).__init__()
        self.transformer = nn.TransformerEncoder(nn.TransformerEncoderLayer(NUM_FEATURES_PER_SAMPLE, NUM_HEADS),
                                                 NUM_LAYERS)
        self.final = nn.Linear(NUM_FEATURES_PER_SAMPLE * SEQUENCE_LENGTH, 2)

    def forward(self, x):
        # this rearranges the dimensions to (seq_length, batch_size, num_features)
        # from (batch_size, seq_length, num_features)
        x = x.permute(2, 0, 1) 

        out = self.transformer(x)
        # print(out.shape)
        out = out.permute(1, 0, 2) 
        out = out.reshape(out.shape[0], -1)
        out = self.final(out)
        return out

## Training

In [26]:
model = Transformer()
loss = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.75)

train_model(model, train, test, loss, optimizer, epochs=500, test_every_n=10, batch_size=512, lr_scheduler=scheduler)

Training Epoch:   0%|          | 0/500 [00:00<?, ?it/s]

Training Epoch:  94%|█████████▍| 470/500 [22:59<01:28,  2.94s/it, Train Loss=0.0222, Test Loss=0.0235]


KeyboardInterrupt: 

## Accuracy Computation

In [27]:
from torcheval.metrics import R2Score
from torch.utils.data import DataLoader

def compute_r2_for(dataset):
    r2 = R2Score()
    train_data = DataLoader(dataset, batch_size=256, shuffle=True)
    for _, data in enumerate(train_data):
        inputs, goal = data[0], data[1]
        outputs = model(inputs)
        r2.update(goal, outputs)
    return r2.compute()

model.to("cpu")
model.eval()
with torch.no_grad():
    train_r2 = compute_r2_for(train)
    print(f"R^2 for training data: {train_r2}")
    test_r2 = compute_r2_for(test)
    print(f"R^2 for testing data: {test_r2}")

R^2 for training data: -5776.265625
R^2 for testing data: 503709.5625


In [20]:
from torcheval.metrics import MeanSquaredError

def compute_mse_for(dataset):
    mse = MeanSquaredError()
    train_data = DataLoader(dataset, batch_size=256, shuffle=True)
    for _, data in enumerate(train_data):
        inputs, goal = data[0], data[1]
        outputs = model(inputs)
        mse.update(outputs, goal)
    return mse.compute()

model.to("cpu")
model.eval()
with torch.no_grad():
    train_mse = compute_mse_for(train)
    print(f"MSE for training data: {train_mse}")
    test_mse = compute_mse_for(test)
    print(f"MSE for testing data: {test_mse}")

MSE for training data: 0.005477847531437874
MSE for testing data: 0.006155923008918762
